In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
w = pd.read_table(p['w_file_path'], index_col=0)

h = pd.read_table(p['h_file_path'], index_col=0)

In [ ]:
kras_w = pd.read_table('../data/kras_map/nmf_k9_w.tsv', index_col=0)

kras_h = pd.read_table('../data/kras_map/nmf_k9_h.tsv', index_col=0)

common_samples = kras_h.columns & h.columns

if 0 < common_samples.size:

    print(common_samples.size)

    ccal.make_comparison_panel(
        h[common_samples],
        kras_h[common_samples],
        name_0=h.index.name,
        name_1='KRAS Program')

In [ ]:
feature_dicts = make_feature_dicts()

In [ ]:
wt_or_h = 'h'

targets = h**0.32

ccal.make_match_panels(
    ((targets).iloc[i] for i in range(targets.shape[0])),
    feature_dicts,
    n_job=a.MAX_N_JOB,
    extreme_feature_threshold=a.EXTREME_FEATURE_THRESHOLD,
    directory_path=p['{}_match_directory_path'.format(wt_or_h)],
    plotly_directory_path=p['{}_match_plotly_directory_path'.format(wt_or_h)],
    overwrite=False)

In [ ]:
n_top = 256

wt_or_h = 'h'

targets = h

for index in targets.index:

    print('=' * 80)

    print(index)

    directory_path = p['{}_match_directory_path'.format(wt_or_h)]

    methylation_scores = pd.read_table(
        os.path.join(directory_path, index, 'Promoter Methylation.tsv'),
        index_col=0)

    methylations = methylation_scores.index[-n_top:]

    methylations = methylations.map(lambda str_: str_.split('_')[0])

    mrna_scores = pd.read_table(
        os.path.join(directory_path, index, 'mRNA.tsv'), index_col=0)

    mrnas = mrna_scores.index[:n_top]

    crispr_scores = pd.read_table(
        os.path.join(directory_path, index, 'Achilles CRISPR.tsv'),
        index_col=0)

    crisprs = crispr_scores.index[:n_top]

    rnai_scores = pd.read_table(
        os.path.join(directory_path, index, 'Achilles RNAi.tsv'), index_col=0)

    rnais = rnai_scores.index[:n_top]

    for (features_0,
         name_0), (features_1,
                   name_1), (features_2, name_2) in itertools.combinations(
                       ((methylations, 'Methylation'), (mrnas, 'mRNA'),
                        (crisprs, 'Achilles CRISPR')), 3):

        print('{} & {} & {}'.format(name_0, name_1, name_2))

        tops = set(features_0 & features_1 & features_2)

        if 0 < len(tops):

            print('\n' * 8)

            target = h.loc[index]

            feature_type_features = {
                'Promoter Methylation': tuple((top, top) for top in tops),
                'mRNA': tuple((top, top) for top in tops),
                'Achilles CRISPR': tuple((top, top) for top in tops),
            }

            for feature_type, features in feature_type_features.items():

                feature_dicts[feature_type]['indices'] = [
                    feature[0] for feature in features
                ]

                feature_dicts[feature_type]['index_aliases'] = [
                    feature[1] for feature in features
                ]

            ccal.make_summary_match_panel(
                target, {
                    feature_type: feature_dicts[feature_type]
                    for feature_type in feature_type_features.keys()
                },
                title='{} Summary'.format(target.name))